In [ ]:
import tensorflow as tf

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path='model/emotion_model.tflite')
interpreter.allocate_tensors()

# Get model input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Model TFLite loaded successfully.")

In [3]:
import cv2
import numpy as np

# Classes for emotion
EMOTION_CLASSES = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

def predict_emotion(frame):
    # Convert to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Resize and normalize
    face_resized = cv2.resize(gray_frame, (48, 48))
    face_resized = face_resized.astype('float32') / 255.0
    input_data = np.expand_dims(face_resized, axis=0)  # Add batch dimension
    input_data = np.expand_dims(input_data, axis=-1)  # Add channel dimension

    # Predict using TFLite model
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    emotion_index = np.argmax(output_data)

    return EMOTION_CLASSES[emotion_index]

In [11]:
# Access camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to access camera.")
    cap.release()
else:
    ret, frame = cap.read()
    cap.release()

    if ret:
        # Predict emotion
        detected_emotion = predict_emotion(frame)
        print(f"Detected Emotion: {detected_emotion}")
    else:
        print("Error: Unable to read frame.")

Detected Emotion: surprise


In [ ]:
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/detect_emotion', methods=['GET'])
def detect_emotion_endpoint():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        return jsonify({'error': 'Unable to access camera.'}), 500

    ret, frame = cap.read()
    cap.release()

    if not ret:
        return jsonify({'error': 'Unable to capture image.'}), 500

    emotion = predict_emotion(frame)
    return jsonify({'emotion': emotion}), 200

# Jalankan API
app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.18.179:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 08:30:32] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 08:30:34] "GET / HTTP/1.1" 404 -


In [14]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# Muat model deteksi ekspresi wajah
model = load_model('emotion_model.h5')  # Pastikan Anda memiliki model deteksi ekspresi wajah

# Muat dataset movies.csv untuk rekomendasi
movies_df = pd.read_csv('dataset/processed_movies.csv')

# Daftar ekspresi yang dapat dikenali
emotion_labels = ['Angry', 'Disguist', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Muat model deteksi wajah
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Fungsi untuk merekomendasikan film berdasarkan ekspresi
def recommend_movies(emotion):
    recommendations = {
        'Happy': ['Comedy', 'Romantic', 'Action'],
        'Sad': ['Comedy','Drama', 'Romantic'],
        'Angry': ['Action', 'Thriller','Comedy'],
        'Fear': ['Horror', 'Mystery'],
        'Surprise': ['Sci-Fi', 'Action','Thriller'],
        'Neutral': ['Adventure', 'Animation'],
        'Disguist': ['Horror', 'Thriller']
    }

    genres = recommendations.get(emotion, ['General'])
    recommended_movies = movies_df[movies_df['genre'].isin(genres)]
    return recommended_movies[['title', 'genre']]

# Fungsi untuk mendeteksi wajah dan ekspresi
def detect_emotion_from_face():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            face = frame[y:y + h, x:x + w]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            face = cv2.resize(face, (48, 48))
            face = face / 255.0
            face = np.expand_dims(face, axis=0)
            face = np.expand_dims(face, axis=-1)

            # Prediksi ekspresi wajah
            emotion_prob = model.predict(face)
            max_index = np.argmax(emotion_prob[0])
            emotion = emotion_labels[max_index]

            # Menampilkan ekspresi wajah dan rekomendasi film
            recommended_movies = recommend_movies(emotion)
            print(f"Detected Emotion: {emotion}")
            print("Recommended Movies:")
            print(recommended_movies)

        cap.release()

# Menjalankan fungsi deteksi ekspresi wajah
if __name__ == "__main__":
    detect_emotion_from_face()